In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.maskers import NiftiLabelsMasker
from nilearn import datasets, image

# === Define Subject IDs ===
# subject_ids = [
#     "sub-0001", "sub-0002", "sub-0003", "sub-0004", "sub-0005",
#     "sub-0006", "sub-0007", "sub-0008", "sub-0009", "sub-0011"
# ]
subject_ids = [
    "sub-0001", "sub-0002"
]

In [2]:
img = nib.load("sub-0001_task-restingstate_acq-seq_bold.nii.gz")
header = img.header
print(header.get_zooms())

(3.0, 3.0, 3.3, 2.0)


In [3]:
# === Construct File Paths ===
fmri_rest_files = [f"{sid}_task-restingstate_acq-seq_bold.nii.gz" for sid in subject_ids]
fmri_task_files = [f"{sid}_task-workingmemory_acq-seq_bold.nii.gz" for sid in subject_ids]
tsv_files = [f"{sid}.tsv" for sid in subject_ids]

In [4]:
# === Load Atlas and Define Masker ===
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100)
masker = NiftiLabelsMasker(atlas.maps, standardize=True)

[get_dataset_dir] Dataset found in C:\Users\elena\nilearn_data\schaefer_2018


In [ ]:
from sklearn.cross_decomposition import CCA
import numpy as np

def compute_trialwise_cca_matrix(trial_tensor):
    """
    trial_tensor: shape (n_trials, n_rois, n_trs)
    Returns: cca_matrix (n_rois x n_rois) with first canonical correlation per ROI pair
    """
    n_trials, n_rois, n_trs = trial_tensor.shape
    cca_matrix = np.zeros((n_rois, n_rois))

    for i in range(n_rois):
        Xi = trial_tensor[:, i, :]  # shape: (n_trials, n_trs)
        for j in range(i + 1, n_rois):
            Xj = trial_tensor[:, j, :]

            cca = CCA(n_components=1)
            Xi_c, Xj_c = cca.fit_transform(Xi, Xj)
            corr = np.corrcoef(Xi_c.T, Xj_c.T)[0, 1]

            cca_matrix[i, j] = corr
            cca_matrix[j, i] = corr

    return cca_matrix

In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import image

def extract_trial_tensor(fmri_path, events_path, masker, n_trs_per_trial=3):
    """
    Εξάγει trial-based time series ανά ROI με σταθερό αριθμό TRs ανά trial.

    Parameters:
        fmri_path (str): path to .nii.gz file (task-based fMRI)
        events_path (str): path to .tsv file (BIDS events)
        masker (NiftiLabelsMasker): για εξαγωγή ROI time series
        n_trs_per_trial (int): πόσοι TRs ανά trial (default = 3)

    Returns:
        trial_tensor: NumPy array (n_trials, n_rois, n_trs_per_trial)
    """
    # === Load fMRI image and get TR ===
    img = nib.load(fmri_path)
    tr = img.header.get_zooms()[3]  # Repetition Time in seconds

    # === Load Events File ===
    events = pd.read_csv(events_path, sep="\t")

    trial_data = []

    for idx, row in events.iterrows():
        onset = row["onset"]
        start_tr = int(np.floor(onset / tr))
        tr_indices = list(range(start_tr, start_tr + n_trs_per_trial))

        # === Extract Trial Image & Mask ===
        trial_img = image.index_img(img, tr_indices)
        trial_ts = masker.fit_transform(trial_img)  # (n_trs, n_rois)
        trial_data.append(trial_ts.T)  # Transpose: (n_rois, n_trs)

    # === Stack Trials into Tensor ===
    trial_tensor = np.stack(trial_data, axis=0)  # (n_trials, n_rois, n_trs)
    return trial_tensor


In [ ]:
trial_tensor = extract_trial_tensor(
    fmri_path="sub-0002_task-workingmemory_acq-seq_bold.nii.gz",
    events_path="sub-0002.tsv",
    masker=masker,
    n_trs_per_trial=3
)

print("Trial tensor shape:", trial_tensor.shape)
# Αναμένεται: (n_trials, 100, 3)

In [ ]:
cca_matrix = compute_trialwise_cca_matrix(trial_tensor)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cca_matrix, cmap="coolwarm", center=0, square=True)
plt.title("CCA Matrix Across ROIs (Trial-Based)")
plt.xlabel("ROI")
plt.ylabel("ROI")
plt.show()